# Fasttext implementation in Gensim
For code examples etc. see the [tutorial](https://github.com/RaRe-Technologies/gensim/blob/de8657e9b8d5192750296b6765175c31c8bb3298/docs/notebooks/FastText_Tutorial.ipynb).
Including two types of warning suppression - check in future installations whether they are still necessary. The first warning only occurs on Windows machines and tells the user that an alternative function will be used. The second warnings line suppresses the "FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated" warning which comes from Numpy.

In [1]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
warnings.filterwarnings(action='ignore', category=FutureWarning, module='gensim' )
import gensim
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText as FT_gensim

In [2]:
file = "corpus/sux_tl.txt"
data = LineSentence(file)

In [4]:
model_gensim = FT_gensim(data, size=100, sg=1, iter=10)
# iter is for epochs

In [5]:
model_gensim.wv.most_similar(positive=["kir₁₁"])

[('u₈', 0.7961888313293457),
 ('{munus}aš₂-gar₃', 0.7794939279556274),
 ('ir₁₁', 0.7784496545791626),
 ('{munus}as₂-gar₃', 0.7783188819885254),
 ('šimašgi₂', 0.7776924967765808),
 ('{munus}si-as₂-gar₃', 0.7603960037231445),
 ('sila₄-nita₂', 0.75836181640625),
 ('dara₄-nita₂', 0.7536314129829407),
 ('dili-ur₄', 0.7372194528579712),
 ('šimašgi', 0.7250467538833618)]

In [ ]:
"nita[male]N" in model_gensim.wv